# Hadronic EoS Generation (Smart Mixing)

## Role in the Project
This script is responsible for generating the "Hadronic" portion of the synthetic dataset. It produces thousands of physically valid Neutron Star Equations of State (EoS) based on standard nuclear physics models.

To ensure the machine learning model is trained on a diverse and representative dataset, this module employs a **Smart Inverse Sampling** technique. Instead of randomly tweaking parameters and hoping for a stable star, the algorithm targets specific maximum masses (filling the "mass gap" between standard neutron stars and black holes) and mathematically derives the required EoS parameters to achieve them.

## Physics and Equations

### 1. Parametric Mixing of EoS Models
To explore the uncertainty space of dense nuclear matter, new EoS curves are generated by mixing two known "parent" models (e.g., SLy and APR) from the library. A mixing weight $w \in [0, 1]$ determines the influence of each parent.

If $\epsilon_A(P)$ and $\epsilon_B(P)$ are the energy density functions of the two parent models, the mixed energy density $\epsilon_{mix}$ at a base pressure $P_{base}$ is approximated by a geometric mean:

$$
\epsilon_{mix}(P_{base}) \approx \epsilon_A(P_{base})^w \cdot \epsilon_B(P_{base})^{1-w}
$$

### 2. Homologous Scaling (The $\alpha$ Factor)
The Tolman–Oppenheimer–Volkoff (TOV) equations exhibit a scaling symmetry. If the pressure and energy density are scaled by a constant factor $\alpha$, the resulting macroscopic mass $M$ and radius $R$ scale inversely by $\sqrt{\alpha}$.

This relationship is used to "tune" the maximum mass of the generated star. If the weighted average maximum mass of the parents is $M_{base}$, and the desired target mass is $M_{target}$, the required scaling factor $\alpha$ is derived as:

$$
\alpha = \left( \frac{M_{base}}{M_{target}} \right)^2
$$

The final EoS relations for the new synthetic star become:

$$
P = \alpha P_{base}
$$

$$
\epsilon(P) = \alpha \cdot \epsilon_{mix}\left( \frac{P}{\alpha} \right)
$$

### 3. Smart Inverse Sampling
Standard random sampling often produces too many low-mass stars and too few high-mass stars ($M > 2.0 M_{\odot}$), which leads to poor training data for massive objects.

To solve this, the algorithm works backward:
1.  A **Target Maximum Mass** is chosen from a uniform distribution (e.g., $2.0$ to $3.0 M_{\odot}$).
2.  The baseline mass $M_{base}$ is calculated from the parent models.
3.  The necessary scaling perturbation $\delta$ is computed:
    $$
    \delta = \frac{M_{target}}{M_{base}} - 1
    $$
4.  If $\delta$ is within physically reasonable limits (the "Smoothness Limit"), the scaling $\alpha$ is applied. This ensures the dataset is populated evenly across the mass spectrum.

### 4. Topological Slope
For every generated star, the slope of the Mass-Radius curve is calculated at key reference masses (e.g., $1.4 M_{\odot}$). This slope is a critical feature for distinguishing stable branches from unstable ones:

$$
\text{Slope} = \frac{dR}{dM} \approx \frac{R(M + h) - R(M)}{h}
$$

Hadronic stars typically exhibit a negative slope ($dR/dM < 0$) on the stable branch, meaning they shrink as they gain mass.

## Algorithm and Calculations

1.  **Model Selection**: Two parent hadronic models are selected at random from the physics library.
2.  **Targeting**: A target maximum mass is selected. The algorithm checks if the parent models can be scaled to reach this target without violating physical bounds (the scaling factor $\alpha$ must not be too extreme).
3.  **Jitter Injection**: To prevent the model from simply memorizing the discrete transitions of the parent models, the crust-core transition pressure is perturbed by random noise ($\pm 20\%$).
4.  **TOV Integration**: The coupled differential equations for Mass, Pressure, and Tidal Deformability are solved from the center of the star to the surface.
5.  **Validation**:
    *   **Causality Check**: The speed of sound $c_s$ is monitored. If $c_s > c$ (speed of light) at low densities, the model is discarded as unphysical.
    *   **Radius Constraint**: Extremely "fluffy" stars (e.g., $R > 14$ km at $1.4 M_{\odot}$) are rejected to match astrophysical observations.
6.  **Feature Extraction**:
    *   The continuous $M-R$ curve is interpolated.
    *   Radius, Tidal Deformability, and Sound Speed are extracted at canonical masses ($1.4, 1.6, 1.8, 2.0 M_{\odot}$).
    *   The curve is labeled as "0" (Hadronic).

## Inputs and Outputs

### Inputs
*   **Baselines**: A dictionary containing the maximum stable mass of every pure parent model (calculated previously).
*   **EoS Library**: The collection of analytic nuclear physics functions.
*   **Target Constraints**: Minimum and maximum target masses defined in the constants file.

### Outputs
*   **Synthetic Dataset**: A list of data vectors. Each vector represents a stable neutron star configuration and includes:
    *   Macroscopic Observables: Mass, Radius, Tidal Deformability.
    *   Microscopic Parameters: Central Pressure, Central Energy Density, Central Sound Speed.
    *   Topological Features: Slopes of the M-R curve.
    *   Label: 0 (Hadronic).

## Connection to the Overall Workflow
This script operates in parallel with the Quark generation worker. Its output is combined with the Quark dataset to form the complete training set for the machine learning classifier. The "Inverse Sampling" technique implemented here is crucial for balancing the dataset, ensuring that the classifier learns to identify massive hadronic stars just as well as typical ones.